<a href="https://colab.research.google.com/github/SrijaG29/offc_dsa_prac/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
app.component.html

In [ ]:
<div class="form-container">
  <h4 >Sample Form:</h4>
  <form (ngSubmit)="onSubmit()">
    <div class="form-ele">
      <label for="name">Name:</label>
      <input type="text" id="name" required [(ngModel)]="name" name="name">
    </div>
    <div class="form-ele">
      <label for="email">Email:</label>
      <input type="text" id="email" required [(ngModel)]="email" name="email">
    </div>
    <div class="form-ele">
      <label for="message">Message:</label>
      <textarea id="message" rows="4" [(ngModel)]="message" name="message"  required></textarea>
    </div>
    <div class="form-ele">
      <button id="Submit" >Submit</button>
    </div>
  </form>

  <div id="container" *ngIf="toShow">
    <app-messages *ngFor="let ele of messages; index as i"
                  [message]="ele"
                  [index]="i"
                  (del)="deleteMessage($event)">
    </app-messages>
  </div>
</div>


app.component.css

In [ ]:
/*.form-container{
  background-color: gray;
  display: flex;
  flex-direction: column;
  margin: 10%;
  width: 80%;
  height: 80%;
}*/

.form-container {
  max-width: 400px;
  margin: auto;
  padding: 20px;
  border: 1px solid #ccc;
  border-radius: 5px;
  background-color: #f7f7f7;
}

h4{
  text-align: center;
}

.form-ele{
  display:flex;
  padding:1%;
  align-items:center;
}

#Submit{
  background-color:green;
}




app.component.ts

In [ ]:
import { Component } from '@angular/core';
import { RouterOutlet } from '@angular/router';
import { FormsModule } from '@angular/forms';
import { CommonModule } from '@angular/common';
import { MessagesComponent } from './messages/messages.component';

@Component({
  selector: 'app-root',
  standalone: true,
  imports: [FormsModule,CommonModule, MessagesComponent],
  templateUrl: './app.component.html',
  styleUrls: ['./app.component.css']
})
export class AppComponent {
  title = 'first-project-angular';
  name:string='';
  email:string='';
  message:string='';
  toShow:boolean = false;
  messages:Array<any>=[];

  onSubmit(){
    this.toShow = true;
    this.messages.push({
      'name':this.name,
      'email':this.email,
      'message':this.message
    })
  }

  deleteMessage(index: number){
    this.messages.splice(index,1);
  }

}


messages.component.html

In [ ]:
<!--<table id="table" >-->
<!--  <tr class="table-row">-->
<!--    <th>S.No</th>-->
<!--    <th>Name</th>-->
<!--    <th>Email</th>-->
<!--    <th>Message</th>-->
<!--  </tr>-->
<!--  <tr class="table-row" >-->
<!--    <td>{{index+1}}</td>-->
<!--    <td>{{message.name}}</td>-->
<!--    <td>{{message.email}}</td>-->
<!--    <td>{{message.message}}</td>-->
<!--  </tr>-->
<!--</table>-->
<div class="message-details">
  <ul>
    <li>Name: {{message.name}}</li>
    <li>Email: {{message.email}}</li>
    <li>Message: {{message.message}}</li>
  </ul>
  <button id="button" (click)="onDelete()">Delete</button>
</div>


messages.component.css

In [ ]:


.message-details{
   margin: 10px;
   padding: 5px;
   border: 1px solid #ccc;
   background-color: #f7f7f7;
   border-radius: 4px;
   word-wrap: break-word;
   white-space: pre-wrap;
}

li{
  list-style-type: none;
  word-wrap: break-word;
  white-space: pre-wrap;
}

#button{
  background-color:green;
  margin:10px;
}


messages.component.ts

In [ ]:
import { Component, Input, Output, EventEmitter } from '@angular/core';


@Component({
  selector: 'app-messages',
  standalone: true,
  imports: [],
  templateUrl: './messages.component.html',
  styleUrls: ['./messages.component.css']
})
export class MessagesComponent {
  @Input() message:any ='';
  @Input() index:number = -1;

  @Output() del = new EventEmitter();
  onDelete(){
    this.del.emit(this.index);
  }
}
